<div class="alert alert-info">

    
## <center> Virtual Team</center>
### <center> Utilizing AutoGen and OpenAI </center>


<br>

**Experiment 1: Round Robin order**
<br>

<br>

    
    
</div>

In [1]:
import autogen

In [2]:
config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG.json")

llm_config = {"config_list": config_list}

In [3]:
# ANSI escape code for green text
GREEN = "\033[92;1m"
BLUE_BOLD = "\033[94;1m"
RESET = "\033[0m"

In [4]:
# Define all agents with valid names

# Customer Proxy Agent
customer_proxy_agent = autogen.ConversableAgent(
    name="Customer_Proxy", 
    system_message="Communicates customer's requirements to the Project Manager.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

# Project Manager - Expense Tracking App Agent
project_manager_agent = autogen.ConversableAgent(
    name="Project_Manager",
    system_message="""Receives customer requirements, coordinates with all engineers, and compiles the total project duration once all phases are completed.
    
    After receiving durations from all phases (Requirements, Design, Implementation, Testing, Documentation):
    - Sum the durations to calculate the total project duration.
    - Report the final project duration to the Customer Proxy in a structured format.

    Report format:
    - List each phase duration and calculate the total project effort.
    - Clearly indicate the completion of the project.""",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)


# Requirement Engineer - Expense Tracking Agent
requirement_engineer_agent = autogen.ConversableAgent(
    name="Requirement_Engineer",  
    
    system_message= """ First, Creates the requirements document that has the requirements for the use-cases of the project requested by the project manager. 
    
                        Then calculate the {effort} to create the requirements document in hours, days, weeks, or months, follow the steps below:
                            - step 1. work = {the estimate of total number of requirements for the use-cases needed for the sofwtare project} 
                            - step 2. productivity rate = {5 requirements completed every day} 
                            - step 3. effort = {work}/{productivity} .
                            
                        The amount of {work} in this phase is measured by the total number of requirements of all use-cases that are created and documented for the software project. """,

    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# System Engineer - Expense Tracking Agent
system_engineer_agent = autogen.ConversableAgent(
    name="System_Engineer",  

    system_message= """ First, Creates the detailed design document based on the requirements document created by the requirement engineer for the requested software project. 

                        The System Engineer - Expense Tracking estimates the number of pages for each design element in design document based on number of requirements in the requirements document.

                        Then, calculate the {effort} to create the design document in hours, days, weeks, or months, follow the steps below:
                            - step 1. work = {the estimate of total number of pages in the design document for the sofwtare project} 
                            - step 2. productivity rate = {5 pages completed every day} 
                            - step 3. effort = {work}/{productivity} .
                            
                        The amount of {work} in this phase is measured by the total number of pages in the design document that are created 
                        based on the total number of requirements in the requirements document created by the requirement engineer for the software project. """,

    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# Software Engineer - Expense Tracking Agent
software_engineer_agent = autogen.ConversableAgent(
    name="Software_Engineer",  
    
    system_message= """ First, Writes complete source code step by step and develops the software based on each design element of the design document created by the system engineer 
                        and requirements document by the requirement engineer. 

                        The Software Engineer estimates the number of source line of code based on each design element in design document in the design document.
    
                        Then, calculate the {effort} to write the source code (SLOC) in hours, days, weeks, or months, follow the steps below:
                            - step 1. work = {the estimate of total number of lines of code (SLOC) that will be created for the sofwtare project} 
                            - step 2. productivity rate = {50 SLOC completed every day} 
                            - step 3. effort = {work}/{productivity} .
                            
                        The amount of {work} in this phase is measured by the total number of lines of code created based on the 
                        total number of pages in the provided design document for the software project. """,
    
    
    
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# Test Engineer Agent
test_engineer_agent = autogen.ConversableAgent(
    name="Test_Engineer",  


    system_message= """ First, Creates the detailed test plan document which has all test cases and executes the test cases develop based on the software written by sofware engineer, 
                        design document created by system engineer and requirements document created by requirement engineer.


                        The Test Engineer estimates the number of test cases in the test plan  based the number of requirements in the requirements document.
    
                        Then, calculate the {effort} to write and execute the test cases in the test plan in hours, days, weeks, or months, follow the steps below:
                            - step 1. work = {the estimate of total number of test cases that will be created adn executed for the sofwtare project} 
                            - step 2. productivity rate = {2 test cases completed every day} 
                            - step 3. effort = {work}/{productivity} .
                            
                        The amount of {work} in this phase is measured by the total number of test cases created and executed based on the 
                        total number of requirements in the provided requirements specification document for the software project. """,
    
    
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="NEVER",
)

# Documentation Engineer Agent
documentation_engineer_agent = autogen.ConversableAgent(
    name="Documentation_Engineer",
    system_message= """
        Creates the detailed documentation, including user guides and technical materials. Based on the requirements, design, and testing documents:
        
        1. Calculates the total number of pages as follows:
           - Total Pages = Requirements Pages + Design Document Pages + Testing Plan Pages.
           
        2. Calculates the effort for the documentation phase:
           - work = total pages calculated
           - productivity rate = 3 pages completed per day
           - effort = work / productivity rate.
        
        Reports the effort to the Project Manager in days for the final project duration calculation.
    """,
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="NEVER",
)


In [5]:
# Initial message from Customer Proxy to Project Manager - Expense Tracking App
customer_message = (
    """"
I want to build a mobile application for expense tracking and splitting bills among friends or groups. The app should allow users to:
1. Create groups, add members, and log expenses.
2. Automatically split bills among group members and track outstanding balances.
3. Provide notifications for due payments and settlements.
4. Display detailed reports of expenses, including filters by category, date, and group.
5. Integrate with payment gateways for settling balances within the app.
"""
)

# Project Manager - Expense Tracking App's prompt to Requirement Engineer - Expense Tracking
project_manager_to_requirement_engineer_prompt = (
    "I have received the customer requirements from the Customer Proxy as the following " + customer_message +
    "List the detailed use cases and requirements for the system"
    "Ensure clarity and completeness in defining each requirement."
    "Please create at least 10 use cases, with a unique name and number each requirement (e.g.,REQ-01,REQ-02) sequentially across all use cases (e.g., UC-001, UC-002) and organize them in a structured format."
    "Also provide  detailed estimates for the amount of work, effort, and productivity required to complete the requirement engineering phase, measured by how many use cases and requirements are documented."
    "Once completed, report back to me so I can proceed with the project plan."
)


# Requirement Engineer - Expense Tracking's response to Project Manager - Expense Tracking App
requirement_engineer_to_project_manager_response = (
    "I have documented the use cases and numbered each requirement as requested. "
    "I have provided the work and effort estimates along with the productivity rate specified for the requirement engineering phase"
    "I am now handing this over back to you to proceed with the next steps in the project plan."
)

# Project Manager - Expense Tracking App's prompt to System Engineer - Expense Tracking
project_manager_to_system_engineer_prompt = (
    "The Requirement Engineer has completed documenting the requirements."
    "Please proceed with creating the detailed design document which includes all design element based on the requirements document created by the requirement engineer for the requested software project" 
    "Also provide detailed estimates for the amount of work, effort, and productivity required to complete the system design phase measured by the number of pages created for the design document."
    "Once completed, report back to me so I can proceed with the next steps in the project plan."
)


# System Engineer - Expense Tracking's response to Project Manager - Expense Tracking App
system_engineer_to_project_manager_response = (
    "I have completed the design document based on the provided requirements docuemnt. "
    "I have provided the work and effort estimates along with the productivity rate specified for the design phase"
    "I am now handing this over back to you to proceed with the next steps in the project plan."

)

# Project Manager - Expense Tracking App's prompt to Software Engineer - Expense Tracking
project_manager_to_software_engineer_prompt = (
    "The System Engineer has completed the architecture design."
    "Please proceed with writing complete source code step by step and develop the software based on each design element of the design document created by the system engineer and each requirements document by the requirement engineer. "
    "Also provide detailed estimates for the amount of work, effort, and productivity required for the development of software phase measured by number of source line of code (SLOC) written."
    "Once completed, report back to me so we can proceed with the next steps in the project plan."


)

# Software Engineer - Expense Tracking's response to Project Manager - Expense Tracking App
software_engineer_to_project_manager_response = (
    "I have completed the software development based on the software written by sofware engineer, design document created by system engineer and requirements document created by requirement engineer." 
    "I have provided the work and effort estimates along with the productivity rate specified for the implementation phase"
    "I am now handing this over back to you to proceed with the next steps in the project plan."

)

# Project Manager - Expense Tracking App's prompt to Test Engineer
project_manager_to_test_engineer_prompt = (
    "The software has been developed."
    "Please proceed with creating the detailed test plan document which has all test cases and executes the test cases develop based on the software written by sofware engineer, design document created by system engineer and requirements document created by requirement engineer."
    "Also provide detailed estimates for the amount of work, effort, and productivity required to complete the testing phase measured by the number of test cases executed."
    "Once completed, report back to me so we can proceed with the next steps in the project plan."
)


# Test Engineer's response to Project Manager - Expense Tracking App

test_engineer_to_project_manager_response = (
    "I have completed the testing phase and verified that the software meets the requirements. "
    "I have provided the work and effort estimates along with the productivity rate specified for the testing phase"
    "I am now handing this over back to you to proceed with the next steps in the project plan."
)

# Project Manager - Expense Tracking App's prompt to Documentation Engineer
project_manager_to_documentation_engineer_prompt = (
    "The testing is complete." 
    "Please proceed with creating the detailed documentation which includes user documentation and training material based on the software written by sofware engineer, design document created by system engineer, and requirements document created by requirement engineer. "
    "Also provide detailed estimates for the amount of work, effort, and productivity required for the documentation phase, measured by the number of pages created."
    "Once completed, report back to me so we can finalize the project."
)

# Documentation Engineer's response to Project Manager - Expense Tracking App
documentation_engineer_to_project_manager_response = (
    "I have completed the documentation, including user documentation and training material. "
    "I have provided the work and effort estimates along with the productivity rate specified for the documentation phase"
    "I am now handing this over back to you to proceed with finalizing the project."
)




In [6]:
# Create a list of agent pairs to define the conversation flow
conversation_flow = [
    (customer_proxy_agent, project_manager_agent, customer_message),
    (project_manager_agent, requirement_engineer_agent, project_manager_to_requirement_engineer_prompt),
    (requirement_engineer_agent, project_manager_agent, requirement_engineer_to_project_manager_response),
    (project_manager_agent, system_engineer_agent, project_manager_to_system_engineer_prompt),
    (system_engineer_agent, project_manager_agent, system_engineer_to_project_manager_response),
    (project_manager_agent, software_engineer_agent, project_manager_to_software_engineer_prompt),
    (software_engineer_agent, project_manager_agent, software_engineer_to_project_manager_response),
    (project_manager_agent, test_engineer_agent, project_manager_to_test_engineer_prompt),
    (test_engineer_agent, project_manager_agent, test_engineer_to_project_manager_response),
    (project_manager_agent, documentation_engineer_agent, project_manager_to_documentation_engineer_prompt),
    (documentation_engineer_agent, project_manager_agent, documentation_engineer_to_project_manager_response),
    # Final report from Project Manager to Customer Proxy
    (project_manager_agent, customer_proxy_agent, "Final project duration summary report."),
]

In [7]:
# Initialize GroupChat
groupchat = autogen.GroupChat(
    agents=[
        customer_proxy_agent,
        project_manager_agent,
        requirement_engineer_agent,
        system_engineer_agent,
        software_engineer_agent,
        test_engineer_agent,
        documentation_engineer_agent,
    ],
    messages=[],
    speaker_selection_method="round_robin",
    max_round=15,
)

In [8]:
# GroupChatManager setup
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)


In [9]:
# Customer Proxy sends the initial message to the Project Manager - Expense Tracking App
groupchat_result = customer_proxy_agent.initiate_chat(
    recipient=project_manager_agent,
    message=customer_message,
)
# Project Manager sends the prompt to the Requirement Engineer
groupchat_result = manager.initiate_chat(
    agent=project_manager_agent,
    recipient=requirement_engineer_agent,
    message=project_manager_to_requirement_engineer_prompt,
)

Customer_Proxy (to Project_Manager):

"
I want to build a mobile application for expense tracking and splitting bills among friends or groups. The app should allow users to:
1. Create groups, add members, and log expenses.
2. Automatically split bills among group members and track outstanding balances.
3. Provide notifications for due payments and settlements.
4. Display detailed reports of expenses, including filters by category, date, and group.
5. Integrate with payment gateways for settling balances within the app.


--------------------------------------------------------------------------------
chat_manager (to Requirement_Engineer):

I have received the customer requirements from the Customer Proxy as the following "
I want to build a mobile application for expense tracking and splitting bills among friends or groups. The app should allow users to:
1. Create groups, add members, and log expenses.
2. Automatically split bills among group members and track outstanding balances.
3.

<div class="alert alert-info">

    
## <center> Virtual Team</center>
### <center> Utilizing AutoGen and OpenAI </center>


<br>

**Experiment 2: Nested Chat**
<br>

<br>

    
    
</div>

In [10]:
# Define nested conversation flow
nested_chat_queue = [
    {"recipient": requirement_engineer_agent, "message": project_manager_to_requirement_engineer_prompt},
    {"recipient": system_engineer_agent, "message": project_manager_to_system_engineer_prompt},
    {"recipient": software_engineer_agent, "message": project_manager_to_software_engineer_prompt},
    {"recipient": test_engineer_agent, "message": project_manager_to_test_engineer_prompt},
    {"recipient": documentation_engineer_agent, "message": project_manager_to_documentation_engineer_prompt},
    # Final step for Project Manager to compile the report
    {"recipient": project_manager_agent, "message": "Please compile the total project duration based on all phases completed."}
]


In [11]:
# To store engineer responses dynamically
engineer_outputs = {}

In [12]:
# Function to print the next speaker
def print_next_speaker(i, queue):
    if i + 1 < len(queue):
        next_speaker = queue[i + 1]['recipient'].name.replace("_", " ")
        print(f"{GREEN}Next Speaker: {next_speaker}{RESET}")
    else:
        print(f"{GREEN}Next Speaker: Project Manager compiles final report to Customer Proxy{RESET}")


In [13]:
# Function to extract only the engineer's work from the chat response
def extract_engineer_work(chat_result):
    # Extract only the response from the agent (output) and ignore the input
    if chat_result and chat_result.chat_history:
        # Assuming the last entry in chat_history is the engineer's response (output)
        return chat_result.chat_history[-1]['content']
    return 'No output available'

In [14]:
# Function to handle nested chat flow and collect responses
def initiate_nested_chats(agent, nested_chat_queue):
    # Customer Proxy initiates the conversation with Project Manager - Expense Tracking App
    customer_proxy_agent.initiate_chat(
        agent=customer_proxy_agent,
        recipient=project_manager_agent,
        message=customer_message
    )
    
    # To accumulate all previous engineers' work
    accumulated_work = ""

    # Nested conversation begins with Project Manager - Expense Tracking App passing on messages to the agents
    for i, chat in enumerate(nested_chat_queue):
        recipient = chat['recipient']
        message = chat['message']

        # If there is previous work, include it in the message for the next engineer
        if accumulated_work:
            message += f"\n\nHere is the combined work from the previous engineers:\n{accumulated_work}"

        # Print the next speaker
        print_next_speaker(i-1, nested_chat_queue)

        print(f"{GREEN}Project Manager initiates chat with {recipient.name.replace('_', ' ')}")
        
        # Initiate chat and capture the output
        output = project_manager_agent.initiate_chat(
            agent=project_manager_agent,
            recipient=recipient,
            message=message
        )

        # Extract only the agent's work from the response (response)
        response = extract_engineer_work(output)

        # Store structured data in the engineer_outputs
        engineer_outputs[recipient.name] = {
            "speaker": project_manager_agent.name,  # The Project Manager - Expense Tracking App is the speaker here
            "recipient": recipient.name,  # The current agent (Requirement Engineer - Expense Tracking, System Engineer - Expense Tracking, etc.)
            "input_message": message,  # The message passed to the agent (including all previous engineers' work)
            "response": response  # Only the engineer's response/work
        }

        # Append the current engineer's work to the accumulated work
        accumulated_work += f"\n\nWork from {recipient.name.replace('_', ' ')}:\n{response}"

    # Once all nested conversations are complete, Project Manager - Expense Tracking App compiles the final plan
    final_message_to_customer = compile_final_project_plan(engineer_outputs)

    # Project Manager - Expense Tracking App sends final message back to Customer Proxy
    print(f"{GREEN}Project Manager sends the final project plan to Customer Proxy{RESET}")
    project_manager_agent.initiate_chat(
        agent=project_manager_agent,
        recipient=customer_proxy_agent,
        message=final_message_to_customer
    )


In [15]:
# Function to compile the final project plan from engineer outputs and print chat log index
def compile_final_project_plan(engineer_outputs):
    final_project_plan = "### Final Project Plan\n\n"
    final_project_plan += "All phases of the project have been completed. Below is the final project plan detailing the work done by each engineer:\n\n"
    
    # Initialize chat log index starting from 1
    chat_log_index = 1
    
    # Add a section for each phase, with speaker, recipient, message, and response
    for engineer, details in engineer_outputs.items():
        final_project_plan += f"{BLUE_BOLD}#### Chat Log {chat_log_index}: {engineer.replace('_', ' ')} Phase{RESET}\n"
        final_project_plan += f"- Speaker: {details['speaker'].replace('_', ' ')}\n"
        final_project_plan += f"- Recipient: {details['recipient'].replace('_', ' ')}\n"
        final_project_plan += f"- Message:\n{details['input_message']}\n\n"
        final_project_plan += f"- Response:\n{details['response']}\n\n"
        
        # Increment the chat log index for the next engineer
        chat_log_index += 1
    
    final_project_plan += "### Conclusion:\n"
    final_project_plan += "The project is ready for deployment, and all necessary phases have been completed successfully."
    
    return final_project_plan


In [16]:
# Start the outer conversation
initiate_nested_chats(
    agent=project_manager_agent,
    nested_chat_queue=nested_chat_queue
)

Customer_Proxy (to Project_Manager):

"
I want to build a mobile application for expense tracking and splitting bills among friends or groups. The app should allow users to:
1. Create groups, add members, and log expenses.
2. Automatically split bills among group members and track outstanding balances.
3. Provide notifications for due payments and settlements.
4. Display detailed reports of expenses, including filters by category, date, and group.
5. Integrate with payment gateways for settling balances within the app.


--------------------------------------------------------------------------------
Next Speaker: Requirement Engineer
Project Manager initiates chat with Requirement Engineer
Project_Manager (to Requirement_Engineer):

I have received the customer requirements from the Customer Proxy as the following "
I want to build a mobile application for expense tracking and splitting bills among friends or groups. The app should allow users to:
1. Create groups, add members, and lo

<div class="alert alert-info">

    
## <center> Virtual Team</center>
### <center> Utilizing LangGraph/Langchain and OpenAI </center>


<br>

**Experiment 3: Multi-agent Sequential Chain**
<br>

<br>

    
    
</div>



## Setup

First, let's install our required packages and set our API keys:

In [17]:
%%capture --no-stderr
%pip install -U langchain langchain_openai langsmith pandas langchain_experimental matplotlib langgraph langchain_core

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-groq 0.1.9 requires langchain-core<0.3.0,>=0.2.26, but you have langchain-core 0.3.13 which is incompatible.
langgraph-checkpoint-sqlite 1.0.3 requires langgraph-checkpoint<2.0.0,>=1.0.8, but you have langgraph-checkpoint 2.0.2 which is incompatible.


In [18]:
import getpass
import os
from dotenv import load_dotenv

def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

## Create Agents

The following helper functions will help create agents. These agents will then be nodes in the graph.

You can skip ahead if you just want to see what the graph looks like.

In [19]:
from langchain.agents import initialize_agent, load_tools
from langchain.prompts import ChatPromptTemplate
from langchain import LLMChain, OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import SequentialChain

# Define OpenAI API Key
import os

# Initialize Language Model with the new ChatOpenAI API
llm = ChatOpenAI(model="gpt-4o")  # Updated model initialization

# Define prompts for each agent using the AGENTS dictionary
AGENTS = {
    "Project Manager": """Receives customer requirements and creates project plan based on outputs of each engineer""",
    
    "Requirement Engineer": """First, Creates the requirements document that has the requirements for the use-cases of the project requested by the project manager. 
    
    Then calculate the effort to create the requirements document in hours, days, weeks, or months, follow the steps below:
        - step 1. work = the estimate of total number of requirements for the use-cases needed for the software project
        - step 2. productivity rate = 5 requirements completed every day
        - step 3. effort = work/productivity .
        
    The amount of work in this phase is measured by the total number of requirements of all use-cases that are created and documented for the software project.
    Current cumulative work details: {cumulative_details}.""",
    
    "System Engineer": """Creates the design document based on the requirements created by the requirement engineer. Estimates the number of pages for each design element in the document.
    
    Calculate the effort to create the design document in hours, days, weeks, or months:
        - step 1. work = estimate of total pages in design document
        - step 2. productivity rate = 5 pages/day 
        - step 3. effort = work/productivity.
        
    Cumulative work and effort details: {cumulative_details}.""",
    
    "Software Engineer": """Writes source code based on each design element from the design document. Estimates the number of lines of code (SLOC) required.
    
    Calculate the effort for SLOC:
        - step 1. work = estimate of total SLOC
        - step 2. productivity rate = 50 SLOC/day
        - step 3. effort = work/productivity.
        
    Current cumulative work and effort details: {cumulative_details}.""",
    
    "Test Engineer": """Creates a detailed test plan and executes test cases based on the software. Estimates the number of test cases.
    
    Calculate the effort for testing:
        - step 1. work = estimate of total test cases
        - step 2. productivity rate = 2 test cases/day
        - step 3. effort = work/productivity.
        
    Current cumulative work and effort details: {cumulative_details}.""",
    
    "Documentation Engineer": """Creates user documentation and training materials based on the software, design, and requirements documents. 
    
    Calculate the effort for documentation:
        - step 1. work = estimate of total documentation pages
        - step 2. productivity rate = 3 pages/day
        - step 3. effort = work/productivity.
        
    Current cumulative work and effort details: {cumulative_details}.""",
    
    "conclusion_prompt":"Now calculate and display the total effort in days for all engineers."
}

CUSTOMER_MESSAGE = """
I want to build a mobile application for expense tracking and splitting bills among friends or groups. The app should allow users to:
1. Create groups, add members, and log expenses.
2. Automatically split bills among group members and track outstanding balances.
3. Provide notifications for due payments and settlements.
4. Display detailed reports of expenses, including filters by category, date, and group.
5. Integrate with payment gateways for settling balances within the app.
"""

C:\Users\soham\AppData\Local\Temp\ipykernel_20352\1772630606.py:11: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o")  # Updated model initialization


# Setting up the SequentialChain implementation


In [20]:
# Define chains for each agent using cumulative details from the previous step
customer_chain = LLMChain(
    llm=llm,
    prompt=ChatPromptTemplate.from_template(CUSTOMER_MESSAGE),
    output_key="customer_data"
)

project_manager_chain = LLMChain(
    llm=llm,
    prompt=ChatPromptTemplate.from_template(AGENTS["Project Manager"]),
    output_key="project_management"
)

requirement_engineer_chain = LLMChain(
    llm=llm,
    prompt=ChatPromptTemplate.from_template(AGENTS["Requirement Engineer"]),
    output_key="requirements"
)

system_engineer_chain = LLMChain(
    llm=llm,
    prompt=ChatPromptTemplate.from_template(AGENTS["System Engineer"]),
    output_key="system_design"
)

software_engineer_chain = LLMChain(
    llm=llm,
    prompt=ChatPromptTemplate.from_template(AGENTS["Software Engineer"]),
    output_key="implementation"
)

testing_engineer_chain = LLMChain(
    llm=llm,
    prompt=ChatPromptTemplate.from_template(AGENTS["Test Engineer"]),
    output_key="testing_results"
)

documentation_engineer_chain = LLMChain(
    llm=llm,
    prompt=ChatPromptTemplate.from_template(AGENTS["Documentation Engineer"]),
    output_key="documentation"
)

conclusion_chain = LLMChain(
    llm=llm,
    prompt=ChatPromptTemplate.from_template(AGENTS["conclusion_prompt"]),
    output_key="total_effort"
)

# Define the sequential chain, ensuring cumulative data is passed to each agent
sequential_chain = SequentialChain(
    chains=[
        customer_chain,
        project_manager_chain,
        requirement_engineer_chain,
        system_engineer_chain,
        software_engineer_chain,
        testing_engineer_chain,
        documentation_engineer_chain,
        conclusion_chain
    ],
    input_variables=["input_data", "cumulative_details"],
    output_variables=[
        "customer_data", "project_management", "requirements",
        "system_design", "implementation", "testing_results",
        "documentation", "total_effort"
    ]
)

# Initialize cumulative_details with customer requirements
cumulative_details = "Customer requirements: " + CUSTOMER_MESSAGE

# Sequentially execute the chain and capture all outputs in cumulative details
final_output = sequential_chain({"input_data": CUSTOMER_MESSAGE, "cumulative_details": cumulative_details})

# Display each output with cumulative details passed along
print("Customer Input:\n", final_output["customer_data"])
print("Project Manager Output:\n", final_output["project_management"])
print("Requirement Engineer Output:\n", final_output["requirements"])
print("System Engineer Output:\n", final_output["system_design"])
print("Software Engineer Output:\n", final_output["implementation"])
print("Test Engineer Output:\n", final_output["testing_results"])
print("Documentation Engineer Output:\n", final_output["documentation"])

C:\Users\soham\AppData\Local\Temp\ipykernel_20352\744217300.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  customer_chain = LLMChain(
C:\Users\soham\AppData\Local\Temp\ipykernel_20352\744217300.py:74: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  final_output = sequential_chain({"input_data": CUSTOMER_MESSAGE, "cumulative_details": cumulative_details})


Customer Input:
 Building a mobile application for expense tracking and bill splitting involves several steps, from planning and design to development and deployment. Here's a high-level overview of how to approach this project:

### 1. Define Requirements and Features

Start by clearly defining the core features and requirements of your app:

- **User Accounts:** Users should be able to sign up, log in, and manage their profiles.
- **Group Management:** Allow users to create groups, invite members, and manage group settings.
- **Expense Logging:** Enable users to add expenses with details like amount, category, date, and description.
- **Bill Splitting:** Automatically split expenses among group members and track who owes whom.
- **Notifications:** Implement push notifications and in-app alerts for payment reminders and settlements.
- **Reports and Analytics:** Provide detailed reports with filters by category, date, and group.
- **Payment Integration:** Integrate with popular payment

<div class="alert alert-info">

    
## <center> Virtual Team</center>
### <center> Utilizing LangGraph/Langchain and OpenAI </center>


<br>

**Experiment 4: Multi-agent graph pipelines**
<br>

<br>

    
    
</div>

In [21]:
import getpass
import os
from dotenv import load_dotenv

def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")


_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

<div class="admonition tip">
    <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for LangGraph development</p>
    <p style="padding-top: 5px;">
        Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started <a href="https://docs.smith.langchain.com">here</a>. 
    </p>
</div>

## Create Agents

The following helper functions will help create agents. These agents will then be nodes in the graph.

You can skip ahead if you just want to see what the graph looks like.

In [22]:
import os
from typing import List, Dict, Any, Tuple
from dotenv import load_dotenv
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseMessage
from langchain_core.messages import HumanMessage, SystemMessage
import langgraph
from langgraph.graph import Graph

# Load environment variables
load_dotenv()

# Initialize the language model
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# Define agent roles and their system messages
AGENTS = {
    "Project Manager": """Receives customer requirements and creates project plan based on outputs of each engineer""",
    
    "Requirement Engineer": """First, Creates the requirements document that has the requirements for the use-cases of the project requested by the project manager. 
    
    Then calculate the effort to create the requirements document in hours, days, weeks, or months, follow the steps below:
        - step 1. work = the estimate of total number of requirements for the use-cases needed for the sofwtare project
        - step 2. productivity rate = 5 requirements completed every day
        - step 3. effort = work/productivity .
        
    The amount of work in this phase is measured by the total number of requirements of all use-cases that are created and documented for the software project.""",
    
    "System Engineer": """First, Creates the detailed design document based on the requirements document created by the requirement engineer for the requested software project. 

    The System Engineer - Expense Tracking estimates the number of pages for each design element in design document based on number of requirements in the requirements document.

    Then, calculate the effort to create the design document in hours, days, weeks, or months, follow the steps below:
        - step 1. work = the estimate of total number of pages in the design document for the sofwtare project 
        - step 2. productivity rate = 5 pages completed every day 
        - step 3. effort = work/productivity .
        
    The amount of work in this phase is measured by the total number of pages in the design document that are created 
    based on the total number of requirements in the requirements document created by the requirement engineer for the software project.""",
    
    "Software Engineer": """First, Writes complete source code and develops the software based on each design element of the design document created by the system engineer 
    and requirements document by the requirement engineer. 

    Second: The Software Engineer estimates the number of source line of code based on each design element from the design document.

    Then, calculate the effort to write the source code (SLOC) in hours, days, weeks, or months, follow the steps below:
        - step 1. work = the estimate of total number of lines of code (SLOC) that will be created for the sofwtare project 
        - step 2. productivity rate = 50 SLOC completed every day
        - step 3. effort = work/productivity .
        
    The amount of work in this phase is measured by the total number of lines of code created based on the 
    total number of pages in the provided design document for the software project.""",
    
    "Test Engineer": """First, Creates the detailed test plan document which has all test cases and executes the test cases develop based on the software written by sofware engineer, 
    design document created by system engineer and requirements document created by requirement engineer.

    The Test Engineer estimates the number of test cases in the test plan  based the number of requirements in the requirements document.

    Then, calculate the effort to write and execute the test cases in the test plan in hours, days, weeks, or months, follow the steps below:
        - step 1. work = the estimate of total number of test cases that will be created adn executed for the sofwtare project
        - step 2. productivity rate = 2 test cases completed every day
        - step 3. effort = work/productivity .
        
    The amount of work in this phase is measured by the total number of test cases created and executed based on the 
    total number of requirements in the provided requirements specification document for the software project.""",
    
    "Documentation Engineer": """First, Creates the detailed documentation which includes user documentation and training material based on the software written by sofware engineer, 
    design document created by system engineer, and requirements document created by requirement engineer. 

    The Documentation Engineer then estimates the number of pages for the documentation by adding the number of requirements in the requirements document, and the
    number of pages in the design document.

    Then, calculate the effort to write documentation pages in the documentation and training phase in hours, days, weeks, or months, follow the steps below:
        - step 1. work = the estimate of total number of documentation pages that will be created and executed for the software project
        - step 2. productivity rate = 3 pages completed every day
        - step 3. effort = work/productivity .
        
    The amount of work in this phase is measured by the total number of documentation pages based on the 
    total number of requirements in the provided requirements specification document for the software project.""",
    "conclusion_prompt":"Now add all the agents output days - Requirement engineer, system engineer, software engineer, test engineer, documentation engineer ."
}

# Define the customer message
CUSTOMER_MESSAGE = """
I want to build a mobile application for expense tracking and splitting bills among friends or groups. The app should allow users to:
1. Create groups, add members, and log expenses.
2. Automatically split bills among group members and track outstanding balances.
3. Provide notifications for due payments and settlements.
4. Display detailed reports of expenses, including filters by category, date, and group.
5. Integrate with payment gateways for settling balances within the app.
"""

# Graph Construction

- Each agent is created as a node in a graph with context tracking.
- The flow follows the waterfall model, with dependencies set between nodes (e.g., Project Manager to Requirement Engineer, Requirement Engineer to System Engineer, etc.).
- A create_agent_node function enables each agent to process inputs and produce outputs sequentially.

In [25]:
# Function to create an agent node that takes the previous output as input
def create_agent_node(agent_name: str):
    prompt = ChatPromptTemplate.from_messages([
        ("system", AGENTS[agent_name]),
        ("human", "{input}"),
    ])
    
    def agent_function(state):
        # Collect the previous context and input message
        previous_output = state.get("context", "")
        agent_input = state.get("input", CUSTOMER_MESSAGE)
        full_input = f"{previous_output}\n\n{agent_input}"  # Accumulate context with new input
        
        # Format and get response
        messages = prompt.format_messages(input=full_input)
        response = llm(messages)
        
        # Update context with current agent's output
        current_output = response.content
        return {"output": current_output, "context": previous_output + "\n" + current_output}
    
    return agent_function

# Create the graph
graph = Graph()

# Add nodes for each agent with context handling
for agent_name in AGENTS.keys():
    graph.add_node(agent_name, create_agent_node(agent_name))

# Define the edges (waterfall model flow)
graph.add_edge("Project Manager", "Requirement Engineer")
graph.add_edge("Requirement Engineer", "System Engineer")
graph.add_edge("System Engineer", "Software Engineer")
graph.add_edge("Software Engineer", "Test Engineer")
graph.add_edge("Test Engineer", "Documentation Engineer")
graph.add_edge("Documentation Engineer", "conclusion_prompt")

# Set the entry point
graph.set_entry_point("Project Manager")

- Compiling all the graph nodes

In [24]:
# Compile the graph
app = graph.compile()

# Initialize state with input and empty context
state = {"input": CUSTOMER_MESSAGE, "context": ""}

# Run the graph and forward outputs to subsequent agents
for output in app.stream(state):
    for agent_name, agent_output in output.items():
        print(f"Agent: {agent_name}")
        print(f"Output: {agent_output['output']}\n")
        
        # Update context for the next agent
        state["context"] = agent_output["context"]

print("Project completed.")

C:\Users\soham\AppData\Local\Temp\ipykernel_20352\3630488793.py:16: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)


Agent: Project Manager
Output: To build a mobile application for expense tracking and splitting bills among friends or groups, we will need to create a comprehensive project plan that outlines the key phases, tasks, and responsibilities of each engineer involved. Below is a high-level project plan based on the requirements provided:

### Project Plan for Expense Tracking and Bill Splitting Mobile App

#### Phase 1: Requirements Gathering and Analysis
- **Duration:** 1 week
- **Tasks:**
  - Conduct meetings with stakeholders to gather detailed requirements.
  - Analyze and document functional and non-functional requirements.
  - Define user personas and use cases.
- **Output:** Requirements Specification Document

#### Phase 2: Design
- **Duration:** 2 weeks
- **Tasks:**
  - **UI/UX Designer:**
    - Create wireframes and mockups for the app's user interface.
    - Design user-friendly navigation and layout for the app.
  - **System Architect:**
    - Design the system architecture, inc